In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [18]:
df = pd.read_csv(r"C:\Users\Jehan_140826\Documents\y3-telco-SON-SH\Data\SON_Fault_Diagnosis_Dataset.csv", parse_dates=["Timestamp"])

In [3]:
df['Timestamp'] = df['Timestamp'].dt.floor('min')

In [4]:
kpis = ['CQI', 'RSRP', 'SINR', 'Throughput']
percentiles = list(range(10, 100, 10)) 

In [5]:
def calc_percentiles(group):
    features = {}
    for kpi in kpis:
        pct_values = np.percentile(group[kpi], percentiles)
        for i, pct in enumerate(percentiles):
            features[f"{kpi}_P{pct}"] = pct_values[i]
    return pd.Series(features)

In [6]:
grouped_df = df.groupby('Timestamp').apply(calc_percentiles).reset_index()

C:\Users\Jehan_140826\AppData\Local\Temp\2\ipykernel_3512\2835695933.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_df = df.groupby('Timestamp').apply(calc_percentiles).reset_index()


In [7]:
grouped_df.head()

,Timestamp,CQI_P10,CQI_P20,CQI_P30,CQI_P40,CQI_P50,CQI_P60,CQI_P70,CQI_P80,CQI_P90,...,SINR_P90,Throughput_P10,Throughput_P20,Throughput_P30,Throughput_P40,Throughput_P50,Throughput_P60,Throughput_P70,Throughput_P80,Throughput_P90
0,2025-04-29 09:10:00,3.0,3.2,4.3,5.0,6.5,8.0,8.7,10.6,12.8,...,14.740982,8.858764,13.726921,31.239649,41.382033,47.318006,50.961808,52.492424,53.178245,68.186774
1,2025-04-29 09:11:00,1.2,2.4,5.0,6.0,7.0,8.2,11.0,12.6,13.8,...,16.319144,8.122943,15.603333,25.785981,31.229672,37.155415,46.413675,48.817462,58.846860,66.131288
2,2025-04-29 09:12:00,2.0,3.0,4.0,6.0,7.0,8.8,10.0,11.0,13.0,...,16.613507,5.984809,15.271228,20.849983,30.723389,37.695510,43.214829,53.265125,59.408140,67.284576
3,2025-04-29 09:13:00,3.0,4.0,5.8,7.0,8.0,9.6,11.0,11.8,13.4,...,16.053616,7.099218,22.081796,27.270359,30.320268,40.543231,44.753848,55.463666,61.525599,70.694863
4,2025-04-29 09:14:00,2.0,4.0,5.0,6.0,7.0,9.0,9.0,11.0,14.0,...,16.930931,8.345532,21.324040,27.110133,34.400393,40.298064,48.224746,58.336581,64.591540,67.617033


In [8]:
label_encoders = {}
for col in ['Fault_Label', 'Severity_Label']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [9]:
label_df = df.groupby('Timestamp')[['Fault_Label', 'Severity_Label']].agg(lambda x: x.mode()[0]).reset_index()

In [10]:
label_df.head()

,Timestamp,Fault_Label,Severity_Label
0,2025-04-29 09:10:00,124,0
1,2025-04-29 09:11:00,124,0
2,2025-04-29 09:12:00,124,0
3,2025-04-29 09:13:00,124,0
4,2025-04-29 09:14:00,124,0


In [11]:
preprocessed_df = pd.merge(grouped_df, label_df, on='Timestamp')

In [12]:
preprocessed_df = pd.merge(grouped_df, label_df, on='Timestamp')

In [13]:
preprocessed_df.head()

,Timestamp,CQI_P10,CQI_P20,CQI_P30,CQI_P40,CQI_P50,CQI_P60,CQI_P70,CQI_P80,CQI_P90,...,Throughput_P20,Throughput_P30,Throughput_P40,Throughput_P50,Throughput_P60,Throughput_P70,Throughput_P80,Throughput_P90,Fault_Label,Severity_Label
0,2025-04-29 09:10:00,3.0,3.2,4.3,5.0,6.5,8.0,8.7,10.6,12.8,...,13.726921,31.239649,41.382033,47.318006,50.961808,52.492424,53.178245,68.186774,124,0
1,2025-04-29 09:11:00,1.2,2.4,5.0,6.0,7.0,8.2,11.0,12.6,13.8,...,15.603333,25.785981,31.229672,37.155415,46.413675,48.817462,58.846860,66.131288,124,0
2,2025-04-29 09:12:00,2.0,3.0,4.0,6.0,7.0,8.8,10.0,11.0,13.0,...,15.271228,20.849983,30.723389,37.695510,43.214829,53.265125,59.408140,67.284576,124,0
3,2025-04-29 09:13:00,3.0,4.0,5.8,7.0,8.0,9.6,11.0,11.8,13.4,...,22.081796,27.270359,30.320268,40.543231,44.753848,55.463666,61.525599,70.694863,124,0
4,2025-04-29 09:14:00,2.0,4.0,5.0,6.0,7.0,9.0,9.0,11.0,14.0,...,21.324040,27.110133,34.400393,40.298064,48.224746,58.336581,64.591540,67.617033,124,0


In [14]:
preprocessed_df.isnull().sum()

Timestamp         0
CQI_P10           0
CQI_P20           0
CQI_P30           0
CQI_P40           0
CQI_P50           0
CQI_P60           0
CQI_P70           0
CQI_P80           0
CQI_P90           0
RSRP_P10          0
RSRP_P20          0
RSRP_P30          0
RSRP_P40          0
RSRP_P50          0
RSRP_P60          0
RSRP_P70          0
RSRP_P80          0
RSRP_P90          0
SINR_P10          0
SINR_P20          0
SINR_P30          0
SINR_P40          0
SINR_P50          0
SINR_P60          0
SINR_P70          0
SINR_P80          0
SINR_P90          0
Throughput_P10    0
Throughput_P20    0
Throughput_P30    0
Throughput_P40    0
Throughput_P50    0
Throughput_P60    0
Throughput_P70    0
Throughput_P80    0
Throughput_P90    0
Fault_Label       0
Severity_Label    0
dtype: int64

In [21]:
print(df["Fault_Label"].unique())
print(df["Severity_Label"].unique())

['Normal' 'ANT+DPD' 'PA+ED+HW' 'DPD' 'ERP+ED' 'ANT' 'PA+ANT' 'ERP+ED+PA'
 'ED+DPD' 'PA+ERP' 'ED+ERP+PA' 'ED+ERP+ANT' 'ANT+DPD+HW' 'PA' 'PA+ED+ANT'
 'DPD+ANT+ED' 'ED' 'ANT+PA' 'ERP+ED+ANT' 'HW+ED' 'HW' 'HW+ANT+ED'
 'ERP+DPD' 'HW+PA' 'ED+ANT+HW' 'HW+ANT' 'DPD+HW' 'ERP+HW+PA' 'DPD+ERP'
 'HW+ANT+DPD' 'ERP' 'PA+HW' 'ANT+PA+DPD' 'ED+DPD+HW' 'PA+ERP+DPD' 'ED+ERP'
 'ANT+HW+ERP' 'ERP+DPD+PA' 'ED+DPD+ANT' 'DPD+ED' 'DPD+PA' 'ERP+PA'
 'DPD+ANT' 'HW+ERP+ANT' 'ANT+HW+PA' 'HW+ERP' 'DPD+HW+ERP' 'PA+ERP+ANT'
 'ED+PA+ERP' 'HW+ED+ERP' 'HW+ED+PA' 'HW+DPD' 'ED+ANT+PA' 'ED+ANT' 'ANT+HW'
 'ERP+ED+DPD' 'ANT+DPD+PA' 'PA+DPD' 'PA+DPD+ERP' 'HW+PA+ERP' 'ERP+HW'
 'ANT+ED' 'ERP+ANT' 'ANT+ERP' 'ED+HW' 'ERP+HW+ED' 'ANT+HW+DPD'
 'PA+ANT+DPD' 'ANT+ERP+DPD' 'DPD+PA+HW' 'PA+ERP+ED' 'DPD+ERP+PA'
 'ERP+HW+DPD' 'PA+HW+DPD' 'ERP+ANT+DPD' 'PA+ED' 'PA+DPD+ED' 'DPD+ED+ANT'
 'ERP+DPD+HW' 'ANT+ERP+HW' 'PA+ED+ERP' 'ANT+ED+DPD' 'ANT+PA+ERP'
 'ERP+ED+HW' 'ERP+DPD+ED' 'DPD+HW+PA' 'ANT+ED+ERP' 'HW+DPD+ERP'
 'DPD+ERP+ED' 'ERP+PA+HW' 'D